In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import scikitplot as skplt
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import KFold

# Carregando base de dados  pré-processada

In [2]:
dataset = pd.read_csv("../../Database/dataBaseWithNER.csv")
dataset = dataset.drop(columns=["Unnamed: 0"])
dataset = dataset.dropna()
targets = np.array(dataset["target"].array)

dataset

,email,target
0,start date hourahead timee cardinall hou...,0
1,service long desk price structure deal quote ...,0
2,start date cardinall hourahead timee card...,0
3,start date hourahead timee cardinall anc...,0
4,cardinall deliverable revenue management marke...,0
...,...,...
33340,bio matrix scientific group symbo bmxg p...,1
33341,cardinall step away hot naked webcam girl liv...,1
33342,need pill increase performance click seroius ...,1
33343,datee final nom inlet hpl eastrans car...,0


In [3]:
emailsText = []
for email in dataset["email"]:
    emailsText.append(email)

del dataset

In [4]:
print(len(emailsText))

33341


# Representação vetorial TFIDF

In [5]:
tfidfVectorizer = TfidfVectorizer(analyzer="word",max_features=2100)

tfidfTransform = tfidfVectorizer.fit_transform(emailsText)

print(tfidfTransform)

  (0, 1593)	0.26470874972054786
  (0, 1098)	0.35510965844285625
  (0, 671)	0.34195431682743727
  (0, 535)	0.3142974755284843
  (0, 1663)	0.2523502664970415
  (0, 249)	0.0882891907360702
  (0, 1904)	0.14585506953003874
  (0, 861)	0.6284245513555586
  (0, 440)	0.21615559903629922
  (0, 1796)	0.23177737406989862
  (1, 1893)	0.12322552160237797
  (1, 432)	0.16667910623163265
  (1, 1405)	0.2023515027069684
  (1, 1910)	0.21821281232441633
  (1, 773)	0.252147976524646
  (1, 1814)	0.24651126769711865
  (1, 2073)	0.1442701571707954
  (1, 1342)	0.19540664510107641
  (1, 1962)	0.21745283397516152
  (1, 809)	0.3089620245082559
  (1, 1457)	0.1585651873255142
  (1, 297)	0.26945510026530634
  (1, 1102)	0.268483606738479
  (1, 105)	0.2304674552193428
  (1, 1514)	0.2375975711797802
  :	:
  (33340, 734)	0.1499328111246709
  (33340, 1274)	0.08454655116352783
  (33340, 1509)	0.09899129231263658
  (33340, 637)	0.15067486614547307
  (33340, 1426)	0.1353816153153684
  (33340, 1339)	0.22427638740937866
  (333

In [6]:
tfidfLabels = tfidfVectorizer.get_feature_names()
tfidfLabels

E:\DevPack\anaconda3\envs\data_science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aa',
 'ability',
 'able',
 'absolutely',
 'abuse',
 'accept',
 'acceptance',
 'accepted',
 'access',
 'according',
 'account',
 'accounting',
 'accuracy',
 'achieve',
 'acquire',
 'acquired',
 'acquisition',
 'acrobat',
 'act',
 'action',
 'active',
 'activity',
 'actual',
 'actually',
 'acy',
 'ad',
 'add',
 'added',
 'adding',
 'addition',
 'additional',
 'additionally',
 'address',
 'administration',
 'adobe',
 'adult',
 'advance',
 'advanced',
 'advantage',
 'advertisement',
 'advertising',
 'advice',
 'advise',
 'advised',
 'advises',
 'advisor',
 'aep',
 'affair',
 'affect',
 'affiliate',
 'affiliated',
 'affordable',
 'age',
 'agency',
 'agenda',
 'agent',
 'aggressive',
 'agree',
 'agreed',
 'agreement',
 'ahead',
 'aid',
 'ail',
 'aimee',
 'air',
 'al',
 'alert',
 'alias',
 'align',
 'allen',
 'allocated',
 'allocation',
 'allow',
 'allowed',
 'allows',
 'alternative',
 'america',
 'amid',
 'amount',
 'amy',
 'analysis',
 'analyst',
 'ancillary',
 'andmanyother',
 'andrew',


In [7]:
dfTfidf = pd.DataFrame(data=tfidfTransform.toarray(), columns=tfidfLabels)

In [8]:
dfTfidf

,aa,ability,able,absolutely,abuse,accept,acceptance,accepted,access,according,...,xanax,xl,xp,yahoo,year,yes,yield,yo,young,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.143700,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33336,0.0,0.0,0.0,0.0,0.0,0.0,0.015667,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.013857,0.0,0.0,0.0,0.0,0.0,0.0
33337,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.207355,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
33338,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
33339,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
dfTfidf = np.array(dfTfidf)
dfTfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
dfTfidf = dfTfidf.reshape((dfTfidf.shape[0], 1, dfTfidf.shape[1]))

print(dfTfidf.shape)

(33341, 1, 2100)


# Visualização de dados com TSNE

In [11]:
# model = TSNE(n_components=2, random_state=0)
# array_red = model.fit_transform(dfTfidf)
#
# df_tsne = pd.DataFrame(array_red)
#
# df_tsne['Target'] = target
# df_tsne_c1 = df_tsne[df_tsne['Target'] == 0]
#
# df_tsne_c2 = df_tsne[df_tsne['Target'] == 1]
#
# plt.scatter(df_tsne_c1[0].array,df_tsne_c1[1].array,marker='o',color='blue')
#
# plt.scatter(df_tsne_c2[0].array,df_tsne_c2[1].array,marker='o',color='red')
#
# plt.title('Dados')
# plt.xlabel('x')
# plt.ylabel('y')
#
# plt.show()

# Validação

In [12]:
foldsAccuracy = []
foldLosses = []

In [13]:
kfold = KFold(n_splits=4, shuffle=True)

In [14]:
foldCount = 1
for train, test in kfold.split(dfTfidf, targets):
    model = keras.models.Sequential([
        keras.layers.LSTM(500, activation='tanh',recurrent_activation='sigmoid',input_shape=[1, dfTfidf.shape[2]]),

        ########## Stacked LSTM
        # keras.layers.LSTM(784, activation='relu', return_sequences=True, input_shape=[1, dfTfidf.shape[2]]),
        # keras.layers.LSTM(250, activation='relu', return_sequences=True),
        # keras.layers.LSTM(90, activation='relu', return_sequences=True),
        # keras.layers.LSTM(30, activation='relu'),

        keras.layers.Dense(len(set(targets)), activation="softmax")
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])

    print('****************************************************')
    print(f'Iniciando treinamento da fold: {foldCount}.')

    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4,mode='min'), tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, restore_best_weights=True)]

    history = model.fit(dfTfidf[train], targets[train], epochs=200, callbacks=callbacks, validation_split=0.05)

    scores = model.evaluate(dfTfidf[test], targets[test], verbose=0)
    print(f'Score fold {foldCount}: {model.metrics_names[0]} de {scores[0]}; {model.metrics_names[1]} de {scores[1]*100}%')

    foldsAccuracy.append(scores[1] * 100)
    foldLosses.append(scores[0])

    foldCount = foldCount + 1

****************************************************
Iniciando treinamento da fold: 1.
Epoch 1/200
743/743 [==============================] - 5s 3ms/step - loss: 0.6912 - accuracy: 0.5485 - val_loss: 0.6886 - val_accuracy: 0.6739 - lr: 0.0100
Epoch 2/200
743/743 [==============================] - 2s 3ms/step - loss: 0.6862 - accuracy: 0.6217 - val_loss: 0.6835 - val_accuracy: 0.6531 - lr: 0.0100
Epoch 3/200
743/743 [==============================] - 2s 3ms/step - loss: 0.6809 - accuracy: 0.7030 - val_loss: 0.6778 - val_accuracy: 0.7458 - lr: 0.0100
Epoch 4/200
743/743 [==============================] - 2s 3ms/step - loss: 0.6750 - accuracy: 0.7539 - val_loss: 0.6712 - val_accuracy: 0.8513 - lr: 0.0100
Epoch 5/200
743/743 [==============================] - 2s 3ms/step - loss: 0.6680 - accuracy: 0.8146 - val_loss: 0.6635 - val_accuracy: 0.8689 - lr: 0.0100
Epoch 6/200
743/743 [==============================] - 2s 3ms/step - loss: 0.6596 - accuracy: 0.8337 - val_loss: 0.6543 - val_accurac

In [15]:
print('****************************************************')
print('Score de cada fold:')
for i in range(0, len(foldsAccuracy)):
    print('****************************************************')
    print(f'--> Fold {i+1}: Loss: {foldLosses[i]} ; Accuracy: {foldsAccuracy[i]}%')

print('****************************************************')
print('Média de accuracy das folds:')
print(f'--> Accuracy: {np.mean(foldsAccuracy)} (+- {np.std(foldsAccuracy)})')
print(f'--> Loss: {np.mean(foldLosses)}')
print('****************************************************')

****************************************************
Score de cada fold:
****************************************************
--> Fold 1: Loss: 0.06222359091043472 ; Accuracy: 97.93666005134583%
****************************************************
--> Fold 2: Loss: 0.0626860111951828 ; Accuracy: 97.73245453834534%
****************************************************
--> Fold 3: Loss: 0.0632193461060524 ; Accuracy: 97.72045612335205%
****************************************************
--> Fold 4: Loss: 0.07025174796581268 ; Accuracy: 97.58847951889038%
****************************************************
Média de accuracy das folds:
--> Accuracy: 97.7445125579834 (+- 0.12449003612622093)
--> Loss: 0.06459517404437065
****************************************************
